# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [162]:
import re
from collections import Counter


def words(text): return re.findall(r'\w+', text.lower())


def load_ngram_model(ngram_file):
    ngram_model = {}
    with open(ngram_file, 'r') as f:
        for line in f:
            parts = line.strip().split('\t')
            context = parts[1]
            word = parts[2]
            count = int(parts[0])
            if context not in ngram_model:
                ngram_model[context] = {}
            ngram_model[context][word] = count
    for i in ngram_model.keys():
        summa = 0
        for j in ngram_model[i].keys():
            summa += ngram_model[i][j]
        ngram_model[i]["_values_sum"] = summa

    return ngram_model


WORDS = Counter(words(open('big.txt').read()))
NGRAM_MODEL = load_ngram_model('bigrams.txt')
N = sum(WORDS.values())


def P_word(word):
    "Frequency probability of `word`."
    return WORDS[word] / N


def P_context(word, context, default=0.001):
    "Probability of word given the `context` of 1 word since I use bigram model"
    if word in NGRAM_MODEL[context].keys():
        return NGRAM_MODEL[context][word] / NGRAM_MODEL[context]["_values_sum"]
    else:
        return 1 / NGRAM_MODEL[context]["_values_sum"]


def correction(word, context=None):
    "Most probable spelling correction for word."
    if context not in NGRAM_MODEL.keys():
        context = None
    return max(candidates(word), key=lambda x: P_word(x) * P_context(x, context) if context else P_word(x))


def candidates(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])


def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)


def edits1(word):
    "All edits that are one edit away from `word`."
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)


def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [149]:
# Example of correction with the use of context info
correction('spors', "big")

'sport'

In [150]:
# Correction without context
correction('spors')

'spurs'

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

I decided to use bigram dataset because of its simplicity and versatility. The words without context(without preceding word) treated the same as in original Norvig’s solution. But if there is preceding word, we calculate the best spelling correction as a product of frequency probability of the word itself and probability of the word given the context. If the context do not have this particular word, it means that it is rare. Therefore, context probability for it would be 1/(total number of words for that context)

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

# Biulding test dataset

In [151]:
# Misspellings dictionary
errors = {}
with open("missp.dat.txt", "r") as file:
    line = file.readline()
    curr_word = None
    while line:
        line = line.lower().strip()
        if line[0] == "$":
            errors[line[1:]] = []
            curr_word = line[1:]
        else:
            errors[curr_word].append(line)
        line = file.readline()

In [152]:
# Example of misspellings dictionary entry
errors["awful"]

['afful',
 'aglift',
 'aleal',
 'alfal',
 'alfall',
 'alfaul',
 'alfol',
 'alfool',
 'alfory',
 'alful',
 'alfule',
 'alfull',
 'allfall',
 'allful',
 'allfull',
 'allfulw',
 'althouge',
 'although',
 'alwafull',
 'alwful',
 'alwy',
 'arfall',
 'arful',
 'arfull',
 'aufal',
 'aufall',
 'aufel',
 'aufful',
 'aufle',
 'auful',
 'aufull',
 'aughfull',
 'aweful',
 'awefull',
 'awfal',
 'awfall',
 'awfull',
 'awlful',
 'awufl',
 'eafall',
 'ofull',
 'olfal',
 'orfal',
 'orfall',
 'orfortil',
 'orful',
 'orfull',
 'orsowl',
 'orthell',
 'ouful']

In [153]:
import random

random.seed(333)

test_set = []
with open("big.txt", "r") as file:
    line = file.readline()
    while line:
        line = line.lower().strip()
        if len(line) > 1:
            splitted = line.split()
            if len(splitted) > 1:
                for i in range(1, len(splitted)):
                    if splitted[i].strip() in errors.keys():
                        context = splitted[i - 1].strip()
                        if not context.isalpha():  # Exclude numerical data
                            continue
                        correct = splitted[i].strip()
                        wrong = random.choice(errors[correct]).strip()
                        test_set.append((context, wrong, correct))
        line = file.readline()

In [154]:
len(test_set)

609224

In [155]:
# Let's decrease the test set to speed up the testing
test_set = random.sample(test_set, 10000)

In [156]:
# Example of test data
test_set[:10]

[('and', 'ro', 'to'),
 ('as', 'as', 'a'),
 ('altercation', 'occureed', 'occurred'),
 ('without', 'nearse', 'any'),
 ('evident', 'wis', 'wish'),
 ('in', 'compy', 'company'),
 ('before', 'hte', 'the'),
 ('the', 'men', 'man'),
 ('on', 't', 'the'),
 ('is', 'beat', 'best')]

# Testing

In [157]:
# Testing my solution
import datetime

begin = datetime.datetime.now()
corrects = 0
for i in test_set:
    result = correction(i[1], i[0])
    if result == i[2]:
        corrects += 1
end = datetime.datetime.now()
print("Accuracy: ", corrects / len(test_set), "Time: ", end - begin)

Accuracy:  0.3173 Time:  0:00:48.262752


In [158]:
# Now let's compare with Norvig's original solution
def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    return WORDS[word] / N


def norwig_correction(word):
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

In [159]:
begin = datetime.datetime.now()
corrects = 0
for i in test_set:
    result = norwig_correction(i[1])
    if result == i[2]:
        corrects += 1
end = datetime.datetime.now()
print("Accuracy: ", corrects / len(test_set), "Time: ", end - begin)

Accuracy:  0.2874 Time:  0:00:36.779969


#### As a result, I was able to achieve greater accuracy in spelling correction by adding the use of the bigram model to the original Norvig's solution. However, the execution time of my solution is ~30% slower, which is expected.

In [160]:
def correct_text_lines(text):
    text = text.lower()
    result = []
    text = text.split("\n")
    for line in text:
        if len(line) == 0:
            result.append(line)
            continue
        splitted = line.split()
        if len(splitted) == 1:
            result.append(correction(splitted[0], None))
            continue

        temp = [correction(splitted[0], None)]
        for i in range(1, len(splitted)):
            temp.append(correction(splitted[i], splitted[i-1]))
        result.append(" ".join(temp))
    return "\n".join(result)


In [175]:
# Example of correction of first few corrupted lines from big.txt
text = "The Praject Gutenberg EBook of The Advitures of Sherlock Holmes\nby Sir Arthur Conan Doyle\n(#15 in out series by Sir Arthur Conan Doyle\n\nCopyright lfaws are changing all over the world. Be sure to heck the\ncopyright laws for your country before downloading or redistributing\nthis or any outher Project Gutenberg eBook.\n\nThis header should be the firt thing seen when viewing this Project\nGutenberg file.  Please do not emove it.  Do not change or edit the\nheadr without writin permission."
corrected = correct_text_lines(text)
print(corrected)

the project gutenberg ebook of the adventures of sherlock holmes
by sir arthur conan doyle
15 in out series by sir arthur conan doyle

copyright laws are changing all over the world be sure to check the
copyright laws for your country before downloading or redistributing
this or any other project gutenberg ebook

this header should be the first thing seen when viewing this project
gutenberg file please do not move it do not change or edit the
head without writing permission
